### Assignment 5: Callbacks

Objective: Practice adding callbacks to Dash apps.

Task:
(1) Build an app that contains the following components user the gapminder dataset: `gdp_pcap.csv`. 

TASK 1 is the same as ASSIGNMENT 4. You are welcome to update your code. 

UI Components:
A dropdown menu that allows the user to select `country`
- The dropdown should allow the user to select multiple countries
- The options should populate from the dataset (not be hard-coded)
A slider that allows the user to select `year`
- The slider should allow the user to select a range of years
- The range should be from the minimum year in the dataset to the maximum year in the dataset
A graph that displays the `gdpPercap` for the selected countries over the selected years
- The graph should display the gdpPercap for each country as a line
- Each country should have a unique color
- The graph should have a title and axis labels in reader friendly format




(2) Write Callback functions for the slider and dropdown to interact with the graph

This means that when a user updates a widget the graph should update accordingly.
The widgets should be independent of each other. 


Layout:
- Use a stylesheet
- There should be a title at the top of the page
- There should be a description of the data and app below the title (3-5 sentences)
- The dropdown and slider should be side by side above the graph and take up the full width of the page
- The graph should be below the dropdown and slider and take up the full width of the page


Submission:
- Deploy your app on Render. 
- In Canvas, submit the URL to your public Github Repo (made specifically for this assignment)
- The readme in your GitHub repo should contain the URL to your Render page. 


**For help you may use the web resources and pandas documentation. No co-pilot or ChatGPT.**

In [1]:
import pandas as pd                 # needed to read in data into dataframes and manipulate
import plotly.express as px         # needed to create line graph
from dash import Dash, dcc, html, Input, Output, callback    # needed to create ui components

df = pd.read_csv('data/clean_data.csv')
# drop the first column
df = df.drop(df.columns[0], axis=1)

min_year = df['Year'].min()
max_year = df['Year'].max()

df.head(10)                                                               # display first few rows of dataframe to confirm it is cleaned properly

,Country,Year,CO2 Emissions (kt),CH4 Emissions (kt),N2O Emissions (kt),NOx Emissions (kt),SO2 Emissions (kt)
0,Albania,1990,3101.66,1143.87,96.10,17.85,0.57
1,Antigua and Barbuda,1990,288.14,98.91,1.58,NaN,2.83
2,Argentina,1990,100867.50,76790.20,51823.68,509.26,79.20
3,Armenia,1990,21615.96,3169.30,169.57,76.59,0.39
4,Australia,1990,278424.66,4975.33,53.48,1620.79,1585.75
5,Austria,1990,62124.70,415.66,14.50,216.37,73.63
6,Azerbaijan,1990,55893.73,13459.35,3772.70,0.08,NaN
7,Bahamas,1990,1894.20,21.00,NaN,NaN,NaN
8,Barbados,1990,1564.22,1662.57,49.60,2.38,NaN
9,Belarus,1990,103689.71,698.34,55.77,1.38,3.50


In [2]:
stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]  # load the CSS stylesheet

app = Dash(__name__, external_stylesheets=stylesheets)  # initialize the app

app.layout = html.Div(
    [
        html.H1("Interactive Gapminder GDP Per Capita Graph"),  # app title at top of page
        html.Div(
            children=[
                html.Div(
                    dcc.Dropdown(df["Country"], id="country-dropdown", placeholder="Select countries", multi=True),     # use dataframe's countries values in dropdown, also set placeholder and allow multi select
                    className="six columns",    # take up half the width, next to range slider
                ),
                html.Div(
                    dcc.RangeSlider(id="year-slider", min=min_year, max=max_year, step=1, value=[min_year, max_year], marks={str(i): str(i) for i in range(min_year, max_year + 1, 20)}),   # create range slider with min and max years defined earlier, set default value, and set marks every 20 years
                    className="six columns",    # take up half the width, next to dropdown
                ),
            ],
            className="row",    # together make dropdown and range slider take up fill width of page in a single row
        ),
        dcc.Graph(id='result-graph'),  # setup graph component with fig defined earlier
    ],
    className="row",    # take up fill width of page and layout into seperate rows
)

@callback(
    Output('result-graph', 'figure'),
    Input('country-dropdown', 'value'),
    Input('year-slider', 'value'))
def update_graph(selected_countries, selected_year_range):
    # filter for selected year range, selected_year_range is [min_year, max_year]
    filtered_df = df[(df['Year'] >= selected_year_range[0]) & (df['Year'] <= selected_year_range[1])]
    
    # if countries are selected, filter for those
    if selected_countries is not None and len(selected_countries) > 0:
        filtered_df = filtered_df[filtered_df['Country'].isin(selected_countries)]  # .isin checks if value is in array

    # create line graph with x axis as the year from df, y as the gdpPercap from df, and group by country into different colors, also set title and axis labels
    fig = px.line(filtered_df, x="Year", y="CO2 Emissions (kt)", color="Country", title="Carbon Dioxide Emissions Over Time by Country")

    return fig

if __name__ == "__main__":
    app.run_server(jupyter_mode="tab", debug=True)      # run the app

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>